In [1]:
%reload_ext autoreload 
%autoreload 2

import tensorflow as tf
import numpy as np
import os
import sys
#currentpath = os.path.dirname(os.path.realpath(__file__))
project_basedir = '..'#os.path.join(currentpath,'..')
sys.path.append(project_basedir)
from matplotlib import pyplot as plt
import random 
import time
from common.utils import Dataset,ProgressBar
from tflearn.data_flow import DataFlow,DataFlowStatus,FeedDictFlow
from tflearn.data_utils import Preloader,ImagePreloader
import scipy
import pandas as pd
import xmltodict
import common
import tflearn
import copy
from config import conf
from cchess import *
from gameplays.game_convert import convert_game,convert_game_value,convert_game_board
import os, shutil
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
from net.net_maintainer import NetMatainer
from net import resnet

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!nvidia-smi | head -n 20

Mon Jul 16 21:23:49 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 45%   77C    P2    97W / 250W |   2124MiB / 11172MiB |     64%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:42:00.0 Off |                  N/A |
| 24%   44C    P8    17W / 250W |    805MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

# a network predict select and move of Chinese chess, with minimal preprocessing

In [3]:
stamp = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))

In [4]:
#data_dir = os.path.join(conf.history_selfplay_dir,stamp)
data_dir = '../data/history_selfplays/2018-06-22_00-44-48/'

In [5]:
if os.path.exists(data_dir):
    print('data_dir already exist: {}'.format(data_dir))
else:
    print('creating data_dir: {}'.format(data_dir))
    os.mkdir("{}".format(data_dir))

data_dir already exist: ../data/history_selfplays/2018-06-22_00-44-48/


In [6]:
GPU_CORE = [1]
BATCH_SIZE = 512
BEGINING_LR = 0.01
#TESTIMG_WIDTH = 500
model_name = 'update_model'

In [7]:
distribute_dir = conf.distributed_datadir
filelist = os.listdir(data_dir)
#filelist = os.listdir(data_dir)
#filelist = [os.path.join(distribute_dir,i) for i in filelist]

In [8]:
network_dir = conf.distributed_server_weight_dir

In [38]:
for f in filelist:
    src = os.path.join(distribute_dir,f)
    dst = os.path.join(data_dir,f)
    shutil.move(src,dst)

In [9]:
filelist = [os.path.join(data_dir,i) for i in filelist]

In [10]:
#filelist = filelist[:1000]
len(filelist)

61653

In [11]:
labels = common.board.create_uci_labels()
label2ind = dict(zip(labels,list(range(len(labels)))))

In [12]:
rev_ab = dict(zip('abcdefghi','abcdefghi'[::-1]))
rev_num = dict(zip('0123456789','0123456789'[::-1]))

In [70]:
class ElePreloader(object):
    def __init__(self,filelist,batch_size=64):
        self.batch_size=batch_size
        #content = pd.read_csv(datafile,header=None,index_col=None)
        self.filelist = filelist#[i[0] for i in content.get_values()]
        self.pos = 0
        self.feature_list = {"red":['A', 'B', 'C', 'K', 'N', 'P', 'R']
                             ,"black":['a', 'b', 'c', 'k', 'n', 'p', 'r']}
        self.batch_size = batch_size
        self.batch_iter = self.iter()
        assert(len(self.filelist) > batch_size)
        #self.game_iterlist = [None for i in self.filelist]
    
    def iter(self):
        retx1,rety1,retx2,rety2 = [],[],[],[]
        vals = []
        filelist = []
        num_filepop = 0
        while True:
            for i in range(self.batch_size):
                filelist = copy.copy(self.filelist)
                random.shuffle(filelist)
                #if self.game_iterlist[i] == None:
                #    if len(filelist) == 0:
                #        filelist = copy.copy(self.filelist)
                #        random.shuffle(filelist)
                #    self.game_iterlist[i] = convert_game_value(filelist.pop(),self.feature_list,None)
                #    num_filepop += 1
                #game_iter = self.game_iterlist[i]
                
                #x1,y1,val1 = game_iter.__next__()
                for one_file in filelist:
                    try:
                        for x1,y1,val1 in convert_game_value(one_file,self.feature_list,None):
                            x1 = np.transpose(x1,[1,2,0])
                            x1 = np.expand_dims(x1,axis=0)

                            #if random.random() < 0.5:
                            #    y1 = [rev_ab[y1[0]],y1[1],rev_ab[y1[2]],y1[3]]
                            #    x1 = x1[:,:,::-1,:]
                            #    #x1 = np.concatenate((x1[:,::-1,:,7:],x1[:,::-1,:,:7]),axis=-1)
                            retx1.append(x1)
                            #rety1.append(y1)
                            oney = np.zeros(len(labels))
                            oney[label2ind[''.join(y1)]] = 1
                            rety1.append(oney)
                            vals.append(val1)

                            if len(retx1) >= self.batch_size:
                                yield (np.concatenate(retx1,axis=0),np.asarray(rety1),np.asarray(vals),num_filepop)
                                retx1,rety1 = [],[]
                                vals = []
                                num_filepop = 0
                    except:
                        print(one_file)
                        import traceback  
                        traceback.print_exc()  
                        continue
                    num_filepop += 1
                    #print(one_file)

    def __getitem__(self, id):
        #pass
        x1,y1,val1,num_filepop = self.batch_iter.__next__()
        return x1,y1,val1,num_filepop
        
    def __len__(self):
        return len(self.filelist)

In [71]:
filelist[0]

'../data/history_selfplays/2018-06-22_00-44-48/2018-06-25_06-40-43_428_mcts-mcts_net-net_w.cbf'

In [72]:
trainset = ElePreloader(filelist=filelist,batch_size=BATCH_SIZE)
with tf.device("/gpu:{}".format(GPU_CORE[0])):
    coord = tf.train.Coordinator()
    trainflow = FeedDictFlow({
            'data':trainset,
        },coord,batch_size=BATCH_SIZE,shuffle=False,continuous=True,num_threads=1)
trainflow.start()

In [76]:
sample_x1,sample_y1,sample_value,sample_num = trainflow.next()['data']
print(sample_num,sample_value)

3 [ 1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1
  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1
  1 -1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  0  0  0 

In [22]:
trainset.filelist[4]

'../data/history_selfplays/2018-06-22_00-44-48/2018-06-22_10-33-21_725_mcts-mcts_net-net_w.cbf'

In [41]:
filepops = []
for sample_x1,sample_y1,sample_value,num_filepop in trainset.iter():
    #xx = x1
    filepops.append(num_filepop)
    print(len(filepops),num_filepop)
    break

1 7
../data/history_selfplays/2018-06-22_00-44-48/2018-06-24_01-17-02_124_mcts-mcts_net-net_w.cbf


Traceback (most recent call last):
  File "<ipython-input-15-7ebe7c9d2c16>", line 50, in iter
    yield (np.concatenate(retx1,axis=0),np.asarray(rety1),np.asarray(vals),num_filepop)
GeneratorExit
Exception ignored in: <generator object ElePreloader.iter at 0x7f4b4193b7d8>
RuntimeError: generator ignored GeneratorExit


complete_number

sample_x1,sample_y1,sample_value = testflow.next()['data']

In [29]:
sample_x1.shape,sample_y1.shape,sample_value.shape

((512, 10, 9, 14), (512, 2086), (512,))

In [30]:
labels[np.argmax(sample_y1[0])]

'h2h5'

In [31]:
np.sum(sample_x1[0],axis=-1)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=uint64)

In [32]:
!mkdir models

mkdir: 无法创建目录"models": 文件已存在


In [33]:
import os
if not os.path.exists("models/{}".format(model_name)):
    os.mkdir("models/{}".format(model_name))

In [34]:
N_BATCH = len(trainset)
#N_BATCH_TEST = 300 * (128 / BATCH_SIZE)


In [35]:
len(trainset)

61653

In [36]:
N_BATCH#,N_BATCH_TEST

61653

In [39]:
latest_netname = NetMatainer(None,network_dir).get_latest()
latest_netname

'2018-06-20_09-00-14'

In [42]:
from net.resnet import get_model

In [43]:
 (sess,graph),((X,training),(net_softmax,value_head,train_op_policy,train_op_value,policy_loss,accuracy_select,global_step,value_loss,nextmove,learning_rate,score)) = \
    get_model('{}/{}'.format(conf.distributed_server_weight_dir,latest_netname),labels,GPU_CORE=GPU_CORE,FILTERS=128,NUM_RES_LAYERS=7,extra=True)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from /work/icybee/icyChessZero/config/../data/prepare_weight/2018-06-20_09-00-14


In [44]:
#with graph.as_default():
#    sess.run(tf.global_variables_initializer())

with graph.as_default():
    train_epoch = 58
    train_batch = 0
    saver = tf.train.Saver(var_list=tf.global_variables())
    saver.restore(sess,"models/{}/model_{}".format(model_name,train_epoch - 1))

In [49]:
train_epoch = 1
train_batch = 0

In [50]:
restore = True
N_EPOCH = 3
DECAY_EPOCH = 20

class ExpVal:
    def __init__(self,exp_a=0.97):
        self.val = None
        self.exp_a = exp_a
    def update(self,newval):
        if self.val == None:
            self.val = newval
        else:
            self.val = self.exp_a * self.val + (1 - self.exp_a) * newval
    def getval(self):
        return round(self.val,2)
    
expacc_move = ExpVal()
exploss = ExpVal()
expsteploss = ExpVal()

begining_learning_rate = 1e-2

pred_image = None
if restore == False:
    train_epoch = 1
    train_batch = 0
for one_epoch in range(train_epoch,N_EPOCH):
    trainset = ElePreloader(filelist=filelist,batch_size=BATCH_SIZE)
    train_epoch = one_epoch
    pb = ProgressBar(worksum=N_BATCH,info=" epoch {} batch {}".format(train_epoch,train_batch))
    pb.startjob()
    
    #for one_batch in range(N_BATCH):
    one_batch = 0
    for batch_x,batch_y,batch_v,one_finish_sum in trainset.iter():
        one_batch += 1
        if pb.finishsum > pb.worksum - 100: # 100 buffer
            break
        
        #batch_x,batch_y,batch_v = trainflow.next()['data']
        batch_v = np.expand_dims(np.nan_to_num(batch_v),1)
        # learning rate decay strategy
        batch_lr = begining_learning_rate * 2 ** -(one_epoch // DECAY_EPOCH)
        with graph.as_default():
            _,step_loss,step_acc_move,step_value = sess.run(
                [train_op_policy,policy_loss,accuracy_select,global_step],feed_dict={
                    X:batch_x,nextmove:batch_y,learning_rate:batch_lr,training:True,
                })
            _,step_value_loss,step_val_predict = sess.run(
                [train_op_value,value_loss,value_head],feed_dict={
                    X:batch_x,learning_rate:batch_lr,training:True,score:batch_v,
                })
            #batch_v = - batch_v
            #batch_x = np.concatenate((batch_x[:,::-1,:,7:],batch_x[:,::-1,:,:7]),axis=-1)
            #_,step_value_loss,step_val_predict = sess.run(
            #    [train_op_value,value_loss,value_head],feed_dict={
            #        X:batch_x,learning_rate:batch_lr,training:True,score:batch_v,
            #    })
            
        
        step_acc_move *= 100
        
        expacc_move.update(step_acc_move)
        exploss.update(step_loss)
        expsteploss.update(step_value_loss)

       
        pb.info = "EPOCH {} STEP {} LR {} ACC {} LOSS {} value_loss {}".format(
            one_epoch,one_batch,batch_lr,expacc_move.getval(),exploss.getval(),expsteploss.getval())
        
        pb.complete(one_finish_sum)
    print()
    with graph.as_default():
        saver = tf.train.Saver(var_list=tf.global_variables())
        saver.save(sess,"../data/models/{}/model_{}".format(model_name,one_epoch))


EPOCH 1 STEP 10708 LR 0.01 ACC 29.18 LOSS 2.7 value_loss 0.48 71.16 % [===================================>---------------] 43873/61653 	 used:6354s eta:2575 s../data/history_selfplays/2018-06-22_00-44-48/_blank


Traceback (most recent call last):
  File "<ipython-input-18-e54fdbcb14dd>", line 34, in iter
    for x1,y1,val1 in convert_game_value(one_file,self.feature_list,None):
  File "../gameplays/game_convert.py", line 68, in convert_game_value
    doc = xmltodict.parse(open(onefile,encoding='utf-8').read())
  File "/usr/local/lib/python3.6/site-packages/xmltodict.py", line 330, in parse
    parser.Parse(xml_input, True)
xml.parsers.expat.ExpatError: no element found: line 1, column 0


EPOCH 1 STEP 15023 LR 0.01 ACC 30.43 LOSS 2.58 value_loss 0.46 99.84 % [=================================================>-] 61554/61653 	 used:8915s eta:14 sss../data/history_selfplays/2018-06-22_00-44-48/2018-06-22_09-07-48_171_mcts-mcts_net-net_peace.cbf



Traceback (most recent call last):
  File "<ipython-input-18-e54fdbcb14dd>", line 50, in iter
    yield (np.concatenate(retx1,axis=0),np.asarray(rety1),np.asarray(vals),num_filepop)
GeneratorExit
Exception ignored in: <generator object ElePreloader.iter at 0x7fa57c0c7888>
RuntimeError: generator ignored GeneratorExit


EPOCH 2 STEP 14936 LR 0.01 ACC 32.84 LOSS 2.37 value_loss 0.43 99.23 % [=================================================>-] 61176/61653 	 used:8924s eta:69 sss../data/history_selfplays/2018-06-22_00-44-48/_blank


Traceback (most recent call last):
  File "<ipython-input-18-e54fdbcb14dd>", line 34, in iter
    for x1,y1,val1 in convert_game_value(one_file,self.feature_list,None):
  File "../gameplays/game_convert.py", line 68, in convert_game_value
    doc = xmltodict.parse(open(onefile,encoding='utf-8').read())
  File "/usr/local/lib/python3.6/site-packages/xmltodict.py", line 330, in parse
    parser.Parse(xml_input, True)
xml.parsers.expat.ExpatError: no element found: line 1, column 0


EPOCH 2 STEP 15023 LR 0.01 ACC 32.55 LOSS 2.41 value_loss 0.46 99.84 % [=================================================>-] 61555/61653 	 used:8976s eta:14 s../data/history_selfplays/2018-06-22_00-44-48/2018-06-20_03-28-26_444_mcts-mcts_net-net_b.cbf



Traceback (most recent call last):
  File "<ipython-input-18-e54fdbcb14dd>", line 50, in iter
    yield (np.concatenate(retx1,axis=0),np.asarray(rety1),np.asarray(vals),num_filepop)
GeneratorExit
Exception ignored in: <generator object ElePreloader.iter at 0x7fa57c0c7888>
RuntimeError: generator ignored GeneratorExit


In [51]:
with graph.as_default():
    saver = tf.train.Saver(var_list=tf.global_variables())
    saver.save(sess,"../data/models/{}_model_{}".format(model_name,one_epoch))

In [53]:
batch_x.shape

(512, 10, 9, 14)

In [54]:
"models/{}/model_{}".format(model_name,one_epoch)

'models/update_model/model_2'

In [55]:
!ls -l 'models/update_model/model_2.data-00000-of-00001'

-rw-rw-r-- 1 meta meta 38213816  5月 30 15:09 models/update_model/model_2.data-00000-of-00001


In [46]:
model_name

'update_model'

In [81]:
for f in ['data-00000-of-00001','meta','index']:
    src = "models/{}/model_{}.{}".format(model_name,one_epoch,f)
    dst = os.path.join(network_dir,"{}.{}".format(stamp,f))
    shutil.copyfile(src,dst)

In [16]:
sorted([i[:-6] for i in os.listdir('data/prepare_weight/') if '.index' in i])[::-1][:2]

['2018-05-24_21-19-55', '2018-05-23']

In [3]:
import os
new_name, old_name = sorted([i[:-6] for i in os.listdir('data/prepare_weight/') if '.index' in i])[::-1][:2]

In [4]:
new_name, old_name

('2018-05-30_18-41-03', '2018-05-30_16-00-13')